In [ ]:
# Analysis of the output of experiments

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime
import os
import pathlib

import pandas as pd
import altair as alt
import numpy as np

import plotly.express as px

alt.data_transformers.disable_max_rows()

# from psp.data. import C, trim_pv, filter_rows, get_max_power_for_time_of_day


def _(df, *args, **kwargs):
    print(len(df))
    display(df.head(*args, **kwargs))

In [ ]:
# It's always annoying to set the working directory: we use an environment variable defined in the Makefile.
CWD = os.environ.get("CWD")
if CWD:
    os.chdir(CWD)

In [ ]:
%pwd

In [ ]:
inferred_meta = pd.read_csv("data/meta_inferred.csv").set_index("ss_id")
_(inferred_meta)

In [ ]:
def error_chart(name, color_scheme):

    df = pd.read_csv(f"exp_results/{name}/errors.csv")
    df["ts"] = pd.to_datetime(df["ts"])
    df = df[df["metric"] == "mae"]
    df = df.join(inferred_meta[["factor"]], on="pv_id")
    df = df.rename(columns={"factor": "capacity"})
    df["pred_ts"] = df["ts"] + df["future"].map(lambda x: pd.Timedelta(minutes=x))
    df["weighted_error"] = df["capacity"] * df["error"]
    # df["ts_hour"] = df["ts"].dt.hour.astype(str) + ":" + df["ts"].dt.minute.astype(str)
    df["future"] = df["future"] / 60.0
    df = df[~df["error"].isnull()]
    _(df)

    df["pred_hour"] = df["pred_ts"].dt.hour * 60 + df["pred_ts"].dt.minute

    df = (
        df[["future", "weighted_error", "pred_hour"]]
        .groupby(["pred_hour", "future"])
        .mean()
        .reset_index()
    )

    _(df)
    df["pred_hour"] = pd.to_timedelta(df["pred_hour"], unit="minute")
    df["date"] = pd.Timestamp(2023, 1, 1)
    df["pred_hour"] = df["date"] + df["pred_hour"]

    _(df)
    #    return

    # FIXME do the aggregation here, not in the chart
    chart = (
        alt.Chart(df)
        .mark_line()
        .encode(
            x=alt.X("hoursminutes(pred_hour)", title="Time *at prediction*"),
            y=alt.Y("weighted_error", title="Error"),
            color=alt.Color(
                "future:O",
                scale=alt.Scale(reverse=True, scheme=color_scheme),
                title="Future (hours) | " + name,
            ),
            # row=alt.Row('ts_hour:N'),
            # column=alt.Column('metric'),
        )
        .properties(
            height=300,
            width=600,
        )
    )
    return chart

In [ ]:
names = ["yesterday", "forest"]  # , 'recent_history_adam']
colors = ["greens", "reds", "blues"]
charts = []
for name, color in zip(names, colors):
    print(name)
    print(color)
    charts.append(error_chart(name, color))
    # display(error_chart(name, color))
import functools

(functools.reduce(lambda x, y: x + y, charts)).resolve_scale(
    color="independent"
).properties(title="")